# Using OpenAI SDK to Access Local API

This notebook demonstrates how to use the OpenAI SDK to interact with a locally running API at http://127.0.0.1:8000/v1.

In [2]:
# Import required libraries
from openai import OpenAI
import os

In [3]:
# Configure the OpenAI client to use the local API
client = OpenAI(
    base_url="http://127.0.0.1:8000/v1",
    api_key="your-api-key-here"  # Replace with your actual API key if required
)

In [3]:
# Test the connection by listing available models
models = client.models.list()
print("Available models:")
for model in models:
    print(f"- {model.id}")

Available models:
- 


In [4]:
## 注意，电影相关的表只有对电影的描述，并没有类型标签
## 系统会自动使用大模型根据电影的描述识别出电影是不是和纯美爱情有关，然后再做统计
def query(q: str):
    chat_completion = client.chat.completions.create(
        model="deepseek_chat",
        messages=[
            {"role": "user", "content": q}
        ],
    )
    import json

    v = json.loads(chat_completion.choices[0].message.content)
    return v["data"][0]

def gen_sql(q: str):
    chat_completion = client.chat.completions.create(
        model="deepseek_chat",
        messages=[
            {"role": "user", "content": q}
        ],
    )
    v = chat_completion.choices[0].message.content
    return v

 ## 注意
 因为后续部分case 大模型会对每条记录做处理，这个数据集也还是比较大的，你需要手工把 movies_metadata.csv 文件中的数据量减少到 1000 条左右, 以便测试或者避免长时间等待。

In [10]:
print(query("帮我统计下打分的用户数量"))

{'user_count': 270896}


In [5]:
print(gen_sql("帮我统计下打分的用户数量"))

-- 加载 ratings.csv 文件
load csv.`/Users/allwefantasy/data/movice/ratings.csv` where header="true" as ratings_table;

-- 统计打分的用户数量
select count(distinct userId) as user_count from ratings_table as output;



In [9]:
## 这里，因为用户的描述是不精准的，比如说包含了安迪的玩具，
## 实际原文是： Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene.
print(query("帮我统计下打分的用户数量,使用 rating表 "))

{'user_count': 270896}


In [7]:
## 这里，因为用户的描述是不精准的，比如说包含了安迪的玩具，
## 实际原文是： Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene.
print(query("帮我找到描述包含了安迪玩具的那个电影。"))

{'id': '862', 'title': 'Toy Story', 'overview': "Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."}


In [29]:
## 注意，电影相关的表只有对电影的描述，并没有类型标签
## 系统会自动使用大模型根据电影的描述识别出电影是不是和纯美爱情有关，然后再做统计
print(query("根据电影的描述，统计下纯美爱情的电影数量。"))


{'romance_count': 0}


In [31]:
## 这里，我们要先通过对overview 字段做NLP分析，找到目标电影集合，然后关联到打分表，再统计打分的用户数量
print(query("帮我找到包含了描述包含了安迪玩具的那个电影，然后给该电影打分的用户数量"))

{'user_count': 3986}


In [ ]:
## 这里，我们要先通过对overview 字段做NLP分析，找到目标电影集合，然后关联到打分表，再统计打分的用户数量
print(query(""))

In [6]:
print(gen_sql("帮我找到包含了描述包含了安迪玩具的那个电影，然后给该电影打分的用户数量"))

-- 加载包含电影描述的表格
load csv.`/Users/allwefantasy/data/movice/movies_metadata.csv` where header="true" as movies_metadata;

-- 加载用户评分的表格
load csv.`/Users/allwefantasy/data/movice/ratings.csv` where header="true" as ratings;

-- 使用大模型辅助函数处理电影描述，找出包含“安迪玩具”的电影
select 
id, 
title, 
overview, 
llm_result(deepseek_chat(llm_param(map(
              "instruction",llm_prompt('

根据下面提供的信息，回答用户的问题。

信息上下文：
```
{0}
```

用户的问题： 电影描述中是否包含“安迪玩具”？
请只输出 “是” 或者 “否”
',array(overview))

)))) as contains_andy_toy
from movies_metadata as filtered_movies;

-- 过滤出包含“安迪玩具”的电影
select id, title from filtered_movies where contains_andy_toy like '%是%' as andy_toy_movies;

-- 统计给这些电影打分的用户数量
select 
r.movieId, 
count(distinct r.userId) as user_count
from ratings as r
join andy_toy_movies as m on r.movieId = m.id
group by r.movieId as user_rating_count;

-- 输出结果
select * from user_rating_count as output;

